In [1]:
import tensorflow as tf

In [2]:
from sklearn.datasets import load_diabetes

In [3]:
diabetes_dataset = load_diabetes()

In [5]:
# Save the Input and Target Variables
from sklearn.model_selection import train_test_split

data = diabetes_dataset['data']
targets = diabetes_dataset['target']

In [6]:
train_data, test_data, train_targets, test_targets = train_test_split(data, targets, test_size=0.1)

In [7]:
print(f"Train Data Shape: {train_data.shape}")
print(f"Test Data Shape: {test_data.shape}")
print(f"Train Target Shape: {train_targets.shape}")
print(f"Test Target Shape: {test_targets.shape}")

Train Data Shape: (397, 10)
Test Data Shape: (45, 10)
Train Target Shape: (397,)
Test Target Shape: (45,)


In [8]:
# Now we construct our Model.

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = tf.keras.Sequential([
    Dense(128, activation='relu', input_shape=(train_data.shape[1],)),
    Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1)
])

In [9]:
# Now we compile our model

model.compile(loss='mse', optimizer='adam', metrics=['mae','accuracy'])

In [10]:
# Define Custom Callback:
class LossAndMetricCallback(tf.keras.callbacks.Callback):
    
    # Print the loss after every second batch in the training set
    def on_train_batch_end(self, batch, logs=None):
        if batch % 2 == 0:
            print(f"\n After batch {batch}, the loss is {logs['loss']:7.2f}")
    
    # Print the loss after each batch in the test set
    def on_test_batch_end(self, batch, logs=None):
        print('\n After batch {}, the loss is {:7.2f}.'.format(batch, logs['loss']))

    # Print the loss and mean absolute error after each epoch
    def on_epoch_end(self, epoch, logs=None):
        print('Epoch {}: Average loss is {:7.2f}, mean absolute error is {:7.2f}.'.format(epoch, logs['loss'], logs['mae']))
    
    # Notify the user when prediction has finished on each batch
    def on_predict_batch_end(self,batch, logs=None):
        print("Finished prediction on batch {}!".format(batch))

In [11]:
# Train the Model
history = model.fit(train_data, train_targets, epochs=20, batch_size=100, callbacks=[LossAndMetricCallback()], verbose=False)


 After batch 0, the loss is 29893.04

 After batch 2, the loss is 28118.86
Epoch 0: Average loss is 28629.21, mean absolute error is  151.34.

 After batch 0, the loss is 27317.28

 After batch 2, the loss is 27987.85
Epoch 1: Average loss is 28496.76, mean absolute error is  150.95.

 After batch 0, the loss is 24041.78

 After batch 2, the loss is 28358.69
Epoch 2: Average loss is 28309.87, mean absolute error is  150.39.

 After batch 0, the loss is 33287.23

 After batch 2, the loss is 29390.03
Epoch 3: Average loss is 28041.65, mean absolute error is  149.56.

 After batch 0, the loss is 25017.11

 After batch 2, the loss is 27015.26
Epoch 4: Average loss is 27634.07, mean absolute error is  148.33.

 After batch 0, the loss is 29015.75

 After batch 2, the loss is 27236.49
Epoch 5: Average loss is 27096.89, mean absolute error is  146.67.

 After batch 0, the loss is 27933.21

 After batch 2, the loss is 27459.71
Epoch 6: Average loss is 26363.26, mean absolute error is  144.39.

In [13]:
# Evaluate the Model
model_eval = model.evaluate(test_data, test_targets, batch_size=10, callbacks=[LossAndMetricCallback()], verbose=False)


 After batch 0, the loss is 16615.33.

 After batch 1, the loss is 21633.04.

 After batch 2, the loss is 19110.87.

 After batch 3, the loss is 19054.55.

 After batch 4, the loss is 19003.15.


In [14]:
# Get predictions from the model

model_pred = model.predict(test_data, batch_size=10,
                           callbacks=[LossAndMetricCallback()], verbose=False)

Finished prediction on batch 0!
Finished prediction on batch 1!
Finished prediction on batch 2!
Finished prediction on batch 3!
Finished prediction on batch 4!


### Application - learning rate scheduler
Let's now look at a more sophisticated custom callback. 

We are going to define a callback to change the learning rate of the optimiser of a model during training. We will do this by specifying the epochs and new learning rates where we would like it to be changed.

First we define the auxillary function that returns the learning rate for each epoch based on our schedule.

In [15]:
# Define the learning rate schedule. The tuples below are (start_epoch, new_learning_rate)

lr_schedule = [(4, 0.03), (7, 0.02), (11, 0.005), (15, 0.007)]

def get_new_epoch_lr(epoch, lr):
    epoch_in_sched = [i for i in range(len(lr_schedule)) if lr_schedule[i][0] == int(epoch)]
    if len(epoch_in_sched) > 0:
        return lr_schedule[epoch_in_sched[0]][1]
    else:
        return lr

In [16]:
# Define the custom callback

class LRScheduler(tf.keras.callbacks.Callback):
    
    def __init__(self, new_lr):
        super(LRScheduler, self).__init__()
        # Add the new learning rate function to our callback
        self.new_lr = new_lr

    def on_epoch_begin(self, epoch, logs=None):
        # Make sure that the optimizer we have chosen has a learning rate, and raise an error if not
        if not hasattr(self.model.optimizer, 'lr'):
              raise ValueError('Error: Optimizer does not have a learning rate.')
                
        # Get the current learning rate
        curr_rate = float(tf.keras.backend.get_value(self.model.optimizer.lr))
        
        # Call the auxillary function to get the scheduled learning rate for the current epoch
        scheduled_rate = self.new_lr(epoch, curr_rate)

        # Set the learning rate to the scheduled learning rate
        tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_rate)
        print('Learning rate for epoch {} is {:7.3f}'.format(epoch, scheduled_rate))

In [17]:
# Build the same model as before

new_model = tf.keras.Sequential([
    Dense(128, activation='relu', input_shape=(train_data.shape[1],)),
    Dense(64,activation='relu'),
    tf.keras.layers.BatchNormalization(),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1)        
])

In [18]:
# Compile the model

new_model.compile(loss='mse',
                optimizer="adam",
                metrics=['mae', 'mse'])

In [19]:
# Fit the model with our learning rate scheduler callback

new_history = new_model.fit(train_data, train_targets, epochs=20,
                            batch_size=100, callbacks=[LRScheduler(get_new_epoch_lr)], verbose=False)

Learning rate for epoch 0 is   0.001
Learning rate for epoch 1 is   0.001
Learning rate for epoch 2 is   0.001
Learning rate for epoch 3 is   0.001
Learning rate for epoch 4 is   0.030
Learning rate for epoch 5 is   0.030
Learning rate for epoch 6 is   0.030
Learning rate for epoch 7 is   0.020
Learning rate for epoch 8 is   0.020
Learning rate for epoch 9 is   0.020
Learning rate for epoch 10 is   0.020
Learning rate for epoch 11 is   0.005
Learning rate for epoch 12 is   0.005
Learning rate for epoch 13 is   0.005
Learning rate for epoch 14 is   0.005
Learning rate for epoch 15 is   0.007
Learning rate for epoch 16 is   0.007
Learning rate for epoch 17 is   0.007
Learning rate for epoch 18 is   0.007
Learning rate for epoch 19 is   0.007
